# Load Data

In [10]:
import pandas as pd 

# columns correspond to unitigs so we must transpose this table
def load_unitig_data(path) -> pd.DataFrame:
    sr = pd.read_csv(path, sep=' ')
    sr.set_index('pattern_id',inplace=True)
    return sr.T[1:]

azm_sr = load_unitig_data("azm_sr_gwas_filtered_unitigs.Rtab")
cfx_sr = load_unitig_data("cfx_sr_gwas_filtered_unitigs.Rtab")
cip_sr = load_unitig_data("cip_sr_gwas_filtered_unitigs.Rtab")

metadata = pd.read_csv('metadata.csv')
metadata.set_index('Sample_ID',inplace=True)


In [11]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            3536 non-null   float64
 1   Country         3785 non-null   object 
 2   Continent       3785 non-null   object 
 3   Beta.lactamase  1927 non-null   object 
 4   Azithromycin    3480 non-null   object 
 5   Ciprofloxacin   3129 non-null   object 
 6   Ceftriaxone     3436 non-null   object 
 7   Cefixime        3405 non-null   object 
 8   Tetracycline    1472 non-null   object 
 9   Penicillin      1465 non-null   object 
 10  NG_MAST         3779 non-null   object 
 11  Group           3786 non-null   int64  
 12  azm_mic         3478 non-null   float64
 13  cip_mic         3088 non-null   float64
 14  cro_mic         3434 non-null   float64
 15  cfx_mic         3401 non-null   float64
 16  tet_mic         1472 non-null   float64
 17  pen_mic         1465 no

In [12]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


    # Null/NA Cleaning in Labels

1. Remove rows with NaN in the labels we are trying to predict: 'azm_sr','cfx_sr', 'cip_sr'

---

Note to Jacob:
If I remove all rows with nulls, then we drop down to ~1k entries. Not ideal. I will remove only the nulls in the target labels. By doing that I was able to preserve ~2800 entries.

Additionally, I can technically replace the nulls in the feature set with averages whether they are continuous or discrete, but since we are going to be building some kind of predictive model later, it would be bad practice to run column averages in df.fillna() before we split our dataset into training and test sets.

Therefore, I am going to split the dataset into a training and test set first.

What do you think?

-Jacob
We could evaluate how to fill the NaN values based on how the data is skewed for each feature we want to predict. 
if skew > 0 -> fill NaN with mean .. there are more 0s than 1s, mean will reflect the distribution of 0s and 1s
else if skew < 0 -> fill NaN with median .. there are more 1s than 0s, using the median ensures our negatively skewed data wont be affected by an uneven distribution of 0s and 1s when we fill it


In [13]:

def impute_cols_by_skew(df, columns):
    for column_name in columns:
        sr_skew = df[column_name].skew()
        impute_value = None
        if sr_skew > 0:
            impute_value = df[column_name].mean()
        elif sr_skew < 0:
            impute_value = df[column_name].median()
        df[column_name].fillna(impute_value, inplace=True)

#metadata.dropna(axis=0, how='any', inplace=True, subset=['azm_sr', 'cfx_sr', 'cip_sr'])

It looks like they're all positively skewed so we can use mean values, lets try continuous for now and see what happens. It was worth trying this though

In [14]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


# Removing Un-used labels (Ground truth)
2. Removing 'Year', and unimportant labels 'cro_sr', 'tet_sr', 'pen_sr'

In [15]:
useless_columns = ['Year', 'cro_sr', 'tet_sr', 'pen_sr']

metadata.drop(labels=useless_columns, axis=1, inplace=True)

In [16]:
metadata.columns

Index(['Country', 'Continent', 'Beta.lactamase', 'Azithromycin',
       'Ciprofloxacin', 'Ceftriaxone', 'Cefixime', 'Tetracycline',
       'Penicillin', 'NG_MAST', 'Group', 'azm_mic', 'cip_mic', 'cro_mic',
       'cfx_mic', 'tet_mic', 'pen_mic', 'log2_azm_mic', 'log2_cip_mic',
       'log2_cro_mic', 'log2_cfx_mic', 'log2_tet_mic', 'log2_pen_mic',
       'azm_sr', 'cip_sr', 'cfx_sr'],
      dtype='object')

# Remove Duplicates

3. Removing Duplicate entries in the dataframe

In [17]:
metadata.drop_duplicates(inplace=True)

# Cleaning non-numeric entries in numeric fields to NaN.

4. Turn Non Numeric Entries in Numeric Columns into NaN
5. Cast all numeric rows into float32
---


Notes: Turning them into NaN for now. Will engineer values for all NaNs after train and test splits are made

In [18]:
numeric_columns = [
'Azithromycin',
'Ciprofloxacin',
'Ceftriaxone',
'Cefixime',
'Tetracycline',
'Penicillin',
'NG_MAST',
'Group',
'azm_mic',
'cip_mic',
'cro_mic',
'cfx_mic',
'tet_mic',
'pen_mic',
'log2_azm_mic',
'log2_cip_mic',
'log2_cro_mic',
'log2_cfx_mic',
'log2_tet_mic',
'log2_pen_mic',
'azm_sr',
'cip_sr',
'cfx_sr',
]

for column in numeric_columns:
    metadata[column] = pd.to_numeric(metadata[column], errors='coerce', downcast="float") #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

In [19]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3113 entries, ERR1549286 to ERR2172346
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         3112 non-null   object 
 1   Continent       3112 non-null   object 
 2   Beta.lactamase  1561 non-null   object 
 3   Azithromycin    2832 non-null   float32
 4   Ciprofloxacin   2608 non-null   float32
 5   Ceftriaxone     2856 non-null   float32
 6   Cefixime        2811 non-null   float32
 7   Tetracycline    1282 non-null   float32
 8   Penicillin      1274 non-null   float32
 9   NG_MAST         2671 non-null   float32
 10  Group           3113 non-null   float32
 11  azm_mic         2863 non-null   float32
 12  cip_mic         2626 non-null   float32
 13  cro_mic         2858 non-null   float32
 14  cfx_mic         2841 non-null   float32
 15  tet_mic         1284 non-null   float32
 16  pen_mic         1278 non-null   float32
 17  log2_azm_mic    2863 no

In [20]:
metadata.head(10)

,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,UK,Europe,NaN,NaN,NaN,0.016,NaN,NaN,NaN,9768.0,...,NaN,9.000000,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549290,UK,Europe,NaN,NaN,NaN,0.004,NaN,NaN,NaN,9768.0,...,NaN,9.000000,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549291,UK,Europe,NaN,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,...,NaN,9.000000,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549288,UK,Europe,NaN,NaN,NaN,0.008,NaN,NaN,NaN,9768.0,...,NaN,9.000000,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549299,UK,Europe,NaN,NaN,NaN,0.012,NaN,NaN,NaN,NaN,...,NaN,9.000000,NaN,-6.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549292,UK,Europe,NaN,NaN,NaN,0.023,NaN,NaN,NaN,9768.0,...,NaN,9.000000,NaN,-5.442222,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549298,UK,Europe,NaN,0.50,NaN,0.094,NaN,NaN,NaN,NaN,...,NaN,-1.000000,NaN,-3.411196,NaN,NaN,NaN,0.0,NaN,NaN
ERR1549300,UK,Europe,NaN,NaN,NaN,0.008,NaN,NaN,NaN,NaN,...,NaN,9.000000,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549295,UK,Europe,NaN,0.75,NaN,0.094,NaN,NaN,NaN,NaN,...,NaN,-0.415038,NaN,-3.411196,NaN,NaN,NaN,0.0,NaN,NaN


# One Hot Encode Categorical Columns

6. Turn Categorical Location entries into numerical representation https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
7. Handle 'Beta.lactamase' special case, as it is discrete with many NaNs. Cannot be engineered in the same way as continuous features. One hot encode like the other discrete features but set to all 0 if NaN

In [21]:
geographic_columns=['Country', 'Continent']
metadata = pd.get_dummies(data=metadata, prefix="Encoded", columns=geographic_columns, dtype=float)
metadata = pd.get_dummies(data=metadata, prefix="Encoded_Beta.lactamase", columns=['Beta.lactamase'], dtype=float)


In [22]:
metadata.head(5)

,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,...,Encoded_Africa,Encoded_America,Encoded_Asia,Encoded_Europe,Encoded_Oceania,Encoded_Beta.lactamase_0,Encoded_Beta.lactamase_1,Encoded_Beta.lactamase_2,Encoded_Beta.lactamase_R,Encoded_Beta.lactamase_S
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,NaN,NaN,0.016,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549290,NaN,NaN,0.004,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549291,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549288,NaN,NaN,0.008,NaN,NaN,NaN,9768.0,1058.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR1549299,NaN,NaN,0.012,NaN,NaN,NaN,NaN,0.0,512.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3113 entries, ERR1549286 to ERR2172346
Data columns (total 98 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Azithromycin              2832 non-null   float32
 1   Ciprofloxacin             2608 non-null   float32
 2   Ceftriaxone               2856 non-null   float32
 3   Cefixime                  2811 non-null   float32
 4   Tetracycline              1282 non-null   float32
 5   Penicillin                1274 non-null   float32
 6   NG_MAST                   2671 non-null   float32
 7   Group                     3113 non-null   float32
 8   azm_mic                   2863 non-null   float32
 9   cip_mic                   2626 non-null   float32
 10  cro_mic                   2858 non-null   float32
 11  cfx_mic                   2841 non-null   float32
 12  tet_mic                   1284 non-null   float32
 13  pen_mic                   1278 non-null   float32
 14

# Split Dataframe into Train and Test

8. To ensure that we can do normalization later as it is good practice to split before normalization

In [24]:
from sklearn.model_selection import train_test_split
train_inputs,test_inputs = train_test_split(metadata, test_size=0.20, random_state=42)   # partition and mix entries
print(train_inputs.shape,":", test_inputs.shape)

(2490, 98) : (623, 98)


# Engineer/Impute Values for NaN numerical values

9. Fill out gaps with skew imputation in numeric columns so we have more data by avoiding dropping all rows with NaN

In [25]:
# apply skew based imputation

targets = numeric_columns
impute_cols_by_skew(train_inputs, targets)
impute_cols_by_skew(test_inputs, targets)




/tmp/ipykernel_1258/1940559347.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(impute_value, inplace=True)
/tmp/ipykernel_1258/1940559347.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [26]:
train_inputs.head(5)

,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,...,Encoded_Africa,Encoded_America,Encoded_Asia,Encoded_Europe,Encoded_Oceania,Encoded_Beta.lactamase_0,Encoded_Beta.lactamase_1,Encoded_Beta.lactamase_2,Encoded_Beta.lactamase_R,Encoded_Beta.lactamase_S
Sample_ID,,,,,,,,,,,,,,,,,,,,,
16933_2#60,1.390346,0.003000,0.008,0.016,7.588545,2.730885,10800.000000,474.0,7.62454,0.003000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
16933_2#61,1.390346,7.060406,0.012,0.016,7.588545,2.730885,11045.000000,496.0,7.62454,7.439204,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10625_6#93,0.250000,32.000000,0.125,0.380,3.000000,4.000000,2958.000000,204.0,0.25000,32.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8727_5#76,16.000000,0.015000,0.008,0.015,0.500000,0.250000,4123.210938,1.0,16.00000,0.015000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17428_6#28,0.047000,0.003000,0.003,0.016,7.588545,2.730885,4330.000000,851.0,0.04700,0.003000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# Normalize Numerical Features 

10. Scale all numerical columns' values to be between 0 and 1

In [27]:
#from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

train_inputs[numeric_columns] = normalizer.fit_transform(train_inputs[numeric_columns])
test_inputs[numeric_columns] = normalizer.fit_transform(test_inputs[numeric_columns])

train_inputs.head(5)

,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,...,Encoded_Africa,Encoded_America,Encoded_Asia,Encoded_Europe,Encoded_Oceania,Encoded_Beta.lactamase_0,Encoded_Beta.lactamase_1,Encoded_Beta.lactamase_2,Encoded_Beta.lactamase_R,Encoded_Beta.lactamase_S
Sample_ID,,,,,,,,,,,,,,,,,,,,,
16933_2#60,0.005373,0.000016,0.003875,0.003751,0.029612,0.021320,0.731126,0.439703,0.007431,0.000031,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
16933_2#61,0.005373,0.110291,0.005876,0.003751,0.029612,0.021320,0.747715,0.460111,0.007431,0.116224,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10625_6#93,0.000918,0.499984,0.062383,0.094774,0.011688,0.031235,0.200149,0.189239,0.000229,0.499992,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8727_5#76,0.062445,0.000203,0.003875,0.003501,0.001922,0.001938,0.279045,0.000928,0.015611,0.000219,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17428_6#28,0.000125,0.000016,0.001375,0.003751,0.029612,0.021320,0.293046,0.789425,0.000031,0.000031,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# EDA STEPS

### Data Stats

1. Showing percentage of rows that are in the 3 different label categories

In [28]:
j=[0,0,0]
samples = metadata.index
for sample in samples:
    if metadata['azm_sr'][sample]:
        j[0] +=1
    if metadata['cfx_sr'][sample]:
        j[1] +=1
    if metadata['cip_sr'][sample]:
        j[2] +=1

print(j[0]/len(samples), "% of samples have resistance to azm")
print(j[1]/len(samples), "% of samples have resistance to cfx")
print(j[2]/len(samples), "% of samples have resistance to cip")


0.17732091230324445 % of samples have resistance to azm
0.08898168968840348 % of samples have resistance to cfx
0.5409572759396081 % of samples have resistance to cip


In [29]:
samples = azm_sr.index

# some random unitig from azm_sr

from random import randint
randomUnitig = azm_sr.columns[randint(0,azm_sr.shape[1])]
j = 0
print(azm_sr.shape)
for i in range(azm_sr.shape[0]):
    if azm_sr[randomUnitig][samples[i]]:
        j += 1

print(randomUnitig,"\npresent in", j/azm_sr.shape[0],"% of azm_sr samples (",j,"/",azm_sr.shape[0],')' )

(3970, 515)
GCCACAGCCAGGCAAACGCCATCAACAGCAGCAG 
present in 0.993198992443325 % of azm_sr samples ( 3943 / 3970 )


In [30]:
train_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2490 entries, 16933_2#60 to 17328_4#17
Data columns (total 98 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Azithromycin              2490 non-null   float32
 1   Ciprofloxacin             2490 non-null   float32
 2   Ceftriaxone               2490 non-null   float32
 3   Cefixime                  2490 non-null   float32
 4   Tetracycline              2490 non-null   float32
 5   Penicillin                2490 non-null   float32
 6   NG_MAST                   2490 non-null   float32
 7   Group                     2490 non-null   float32
 8   azm_mic                   2490 non-null   float32
 9   cip_mic                   2490 non-null   float32
 10  cro_mic                   2490 non-null   float32
 11  cfx_mic                   2490 non-null   float32
 12  tet_mic                   2490 non-null   float32
 13  pen_mic                   2490 non-null   float32
 14

In [31]:
test_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 623 entries, 10356_1#54 to SRR2736259
Data columns (total 98 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Azithromycin              623 non-null    float32
 1   Ciprofloxacin             623 non-null    float32
 2   Ceftriaxone               623 non-null    float32
 3   Cefixime                  623 non-null    float32
 4   Tetracycline              623 non-null    float32
 5   Penicillin                623 non-null    float32
 6   NG_MAST                   623 non-null    float32
 7   Group                     623 non-null    float32
 8   azm_mic                   623 non-null    float32
 9   cip_mic                   623 non-null    float32
 10  cro_mic                   623 non-null    float32
 11  cfx_mic                   623 non-null    float32
 12  tet_mic                   623 non-null    float32
 13  pen_mic                   623 non-null    float32
 14 

## Calculate mean, median, mode, variance, and standard deviation

## Univariate Analysis (Distr. of individual variables)

* Identify shape of feature distributions, their skew and whatnot. 
* Potentially apply some pure function to re-engineer the distribution to look more like a normal distribution if needed

## Multivariate Analysis (Show relationship of individual variables wrt to labels, pair plots and such) 

* Use to identify list of impactful features to use in model step later

## Correlation Analysis (Pearson, Spearman, etc)

## Outlier Detection (box plots, then remove outliers)